In [2]:
import pandas as pd
import numpy as np
import pickle as pickle
import re
import nltk
from googletrans import Translator
# import time
import emoji
from tqdm import tqdm

In [3]:
with open('output/frjson.p','br') as f:
    tweets=pickle.load(f)

In [4]:
tweets_pd =pd.DataFrame()

In [5]:
def get_full_text(tweet):
    try:
        return(tweet["extended_tweet"]["full_text"])
    except:
        return(tweet["text"])

In [6]:
tweets_pd['text']=list(map(lambda tweet: get_full_text(tweet), tweets))

In [7]:
pollution = ["air #pollution", "空气污染", "#airpollution", "वायु प्रदुषण", "la pollution de l'air", "la #pollution",
            "air #pollution", "l'air #pollution", "वायु प्रदुषण",'air quality','smog','embouteillage',"#वायु प्रदुषण","वायु #प्रदूषण"]

In [8]:
pollution_re          = re.compile(r'('+'|'.join(pollution)+')',  re.IGNORECASE)

In [9]:
tweets_pd['pollution'] = list(map(lambda tweet: True if pollution_re.search(tweet) else False,tweets_pd.text))  

In [10]:
tweets_pd =tweets_pd[tweets_pd.pollution==True]

In [11]:
tweets_pd.text[370]

"Après avoir traversé l'Atlantique à la #nage, le Français Benoît #Lecomte, 51 ans, traversera cette fois-ci le #Pacifique par la seule force de ses bras pour alerter sur la #pollution des #océans envahis de #plastique. Départ du #Japon mardi https://t.co/DFOyymRG9j"

In [12]:
# translate
df = tweets_pd

In [13]:
df.reset_index(inplace=True)

In [14]:
df.to_csv("output/tempdata_freqword.csv")

In [15]:
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.co.kr',
    ])
trans_text=dict()

In [16]:
def trans(error_index=None):
    if error_index: 
        new_error_index=[]
        for idx in tqdm(error_index):
            text=emoji.demojize(df.text[idx])
            try:
                trans = translator.translate(text,dest='en',src='auto')
                trans_text[idx]=trans.text
            except:
                new_error_index.append(idx)
    
    else:
        new_error_index=[]
        for i in tqdm(range(len(df.text))):
            text = emoji.demojize(df.text[i])
            try:
                trans=translator.translate(text,dest='en',src='auto')
                trans_text[i]=trans.text
            except:
                new_error_index.append(i)
        
    return new_error_index

In [17]:
error=trans()

100%|██████████| 9375/9375 [54:51<00:00,  2.85it/s]


In [19]:
new_error=trans(error)

100%|██████████| 251/251 [01:13<00:00,  3.43it/s]


In [21]:
len(trans_text)

9375

In [29]:
df['trans_text']=list(trans_text.values())

In [31]:
df.to_csv("output/france_freqent_need_process.csv")

In [32]:
from nltk.tokenize import word_tokenize
import re
 
regex_str = [
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens =  [ token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'a n', 'example', '!', ':D', 'http://example.com', '#NLP']


['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':', 'D', 'http://example.com', '#NLP']


In [33]:
from nltk.corpus import stopwords

In [52]:
import string
def remove_stop_words(tweet):
    remove_regex = [    
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
#     r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
]
    junk = re.compile("("+"|".join(remove_regex)+")")
    punctuation = list(string.punctuation)
    stop = stopwords.words('english') + punctuation + ['rt', 'via','RT','We','A','a','of','the','to','is'
                                                       'I','’','The','️','There','If',
                                                       "It's",'You',"also",'so',
                                                       'in','if','It','In','I'
                                                      ]
    
    terms_stop = [term for term in preprocess(tweet) if term not in stop and not junk.search(term)]
    
    return(" ".join(terms_stop))

In [53]:
remove_stop_words("it")

''

In [54]:
from collections import Counter

In [55]:
alltext=" ".join(list(map(lambda tweet: remove_stop_words(tweet),tweets_pd['trans_text'])))

In [56]:
top_100=Counter(alltext.split()).most_common()[0:100]

In [57]:
top_100_pd=pd.DataFrame(top_100)

In [58]:
top_100_pd=top_100_pd.rename(mapper={0:'word',1:'frequency'},axis=1)

In [59]:
top_100_pd

,word,frequency
0,#pollution,3787
1,pollution,3615
2,air,3294
3,France,1200
4,see,872
5,Air,869
6,#plastic,779
7,fight,599
8,traffic,595
9,Smog,528


In [62]:
top_100_pd.to_csv("
                  output/top_100_freq_france.csv",index=False)